## Question 1: Fiction Box – Monthly Revenue & Cancellation Cohort Analysis

In [39]:
import pandas as pd
from openpyxl import Workbook


In [40]:
charges = pd.read_excel("Charges Processed.xls")
subs = pd.read_excel("Subscriptions_by creation date.xls")

print("Data loaded successfully")
print(f"Charges shape: {charges.shape}")
print(f"Subscriptions shape: {subs.shape}")


Data loaded successfully
Charges shape: (3187, 50)
Subscriptions shape: (596, 29)


In [41]:
# Convert date columns
charges["scheduled_at"] = pd.to_datetime(charges["scheduled_at"], errors="coerce")
subs["created_at"] = pd.to_datetime(subs["created_at"], errors="coerce")
subs["cancelled_at"] = pd.to_datetime(subs["cancelled_at"], errors="coerce")

# Keep only successful charges
charges = charges[charges["status"] == "SUCCESS"].copy()

print(f"Successful charges: {len(charges)}")


Successful charges: 3086


In [42]:
max_date = charges["scheduled_at"].max()
three_months_ago = max_date - pd.DateOffset(months=3)

charges_3m = charges[charges["scheduled_at"] >= three_months_ago].copy()

print(f"Charges in last 3 months: {len(charges_3m)}")


Charges in last 3 months: 3083


In [43]:
# Fiction Box subscriptions
fiction_subs = subs[subs["sku"] == "ABOS4FICTION|SUB"].copy()

# Fiction Box charges (last 3 months only)
fiction_charges = charges_3m[
    charges_3m["line_item_sku"] == "ABOS4FICTION|SUB"
].copy()

print(f"Fiction Box subscriptions: {len(fiction_subs)}")
print(f"Fiction Box charges (last 3 months): {len(fiction_charges)}")


Fiction Box subscriptions: 36
Fiction Box charges (last 3 months): 643


In [44]:
fiction_subs["cohort_month"] = fiction_subs["created_at"].dt.to_period("M").astype(str)
fiction_charges["charge_month"] = fiction_charges["scheduled_at"].dt.to_period("M").astype(str)


In [45]:
fiction_data = fiction_charges.merge(
    fiction_subs[["subscription_id", "cohort_month", "cancelled_at"]],
    left_on="line_item_subscription_id",
    right_on="subscription_id",
    how="inner"
)

print(f"Merged Fiction cohort data rows: {len(fiction_data)}")


Merged Fiction cohort data rows: 47


In [46]:
revenue_cohort = (
    fiction_data
    .groupby(["cohort_month", "charge_month"])["store_total_price"]
    .sum()
    .unstack(fill_value=0)
)

print("Question 1 — Revenue Cohort Table")
revenue_cohort


Question 1 — Revenue Cohort Table


charge_month,2025-10,2025-11,2025-12
cohort_month,,,
2025-10,156.1,68.66,94.95
2025-11,0.0,154.04,96.95
2025-12,0.0,0.00,282.93


In [47]:
cancellation_summary = (
    fiction_subs
    .groupby("cohort_month")
    .agg(
        total_subscriptions=("subscription_id", "count"),
        cancelled_subscriptions=("cancelled_at", lambda x: x.notna().sum())
    )
    .reset_index()
)

cancellation_summary["cancellation_rate_percent"] = (
    cancellation_summary["cancelled_subscriptions"]
    / cancellation_summary["total_subscriptions"] * 100
).round(2)

print("Question 1 — Cancellation Cohort Analysis")
cancellation_summary


Question 1 — Cancellation Cohort Analysis


,cohort_month,total_subscriptions,cancelled_subscriptions,cancellation_rate_percent
0,2025-10,11,7,63.64
1,2025-11,8,2,25.00
2,2025-12,17,4,23.53


## Question 2:  Product Performance — Top & Underperforming Products

In [48]:
product_revenue = (
    charges_3m
    .groupby("line_item_sku")
    .agg(
        total_revenue=("store_total_price", "sum"),
        total_charges=("charge_id", "count"),
        unique_subscribers=("line_item_subscription_id", "nunique")
    )
    .reset_index()
    .sort_values("total_revenue", ascending=False)
)

print(f"Total products analyzed: {len(product_revenue)}")
product_revenue.head()


Total products analyzed: 100


,line_item_sku,total_revenue,total_charges,unique_subscribers
47,ABOS3HORRORPLUS,10665.84,366,363
74,ABOS4FICTION|SUB,10146.58,643,489
89,ABOS4MIXED|SUB,4934.83,301,236
70,ABOS4CMT|SUB,4218.96,182,143
50,ABOS4BYOB,3015.19,116,87


In [49]:
product_revenue.tail(10)


,line_item_sku,total_revenue,total_charges,unique_subscribers
21,ABOS2HORRORBASIC|APFREE,18.99,1,1
2,ABOS1SCI|DISCOVERY,18.68,2,1
3,ABOS1THEBOX,16.99,1,1
72,ABOS4FAN|SD12,15.00,1,1
37,ABOS2SCI|SUB,14.00,1,1
31,ABOS2LR|CAMPAIGN|MHTBFREE,13.50,1,1
29,ABOS2KIDS-GA,11.67,1,1
14,ABOS2CMT|SUB_OFFER,10.00,1,1
35,ABOS2ROMANCE|CAMPAIGN,10.00,1,1
63,ABOS4CMTPREWD,0.00,1,1


In [50]:
wb = Workbook()
wb.remove(wb.active)

# Revenue Cohort Sheet
ws1 = wb.create_sheet("Revenue Cohort")
ws1.append(["Cohort"] + list(revenue_cohort.columns))
for idx, row in revenue_cohort.iterrows():
    ws1.append([idx] + list(row))

# Cancellation Sheet
ws2 = wb.create_sheet("Cancellation Cohort")
ws2.append(cancellation_summary.columns.tolist())
for row in cancellation_summary.values.tolist():
    ws2.append(row)

# Product Performance Sheet
ws3 = wb.create_sheet("Product Performance")
ws3.append(product_revenue.columns.tolist())
for row in product_revenue.values.tolist():
    ws3.append(row)

wb.save("Assignment_Results.xlsx")

print("Results exported to Assignment_Results.xlsx")


Results exported to Assignment_Results.xlsx
